# Optimización de Ubicación de Sensores LoRa
## Set Cover Problem mediante Programación Lineal Entera (PLE)

**Objetivo:** Minimizar el número de nodos sensores requeridos ($N_{\text{óptimo}}$) para garantizar el 100% de cobertura en un campo rectangular.

---

## 1. Carga de Configuración y Librerías

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from pulp import *
from datetime import datetime

# Configurar matplotlib para mejor visualización
plt.rcParams['figure.figsize'] = (12, 10)
plt.rcParams['font.size'] = 10

print("Librerías cargadas exitosamente")
print(f"Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Cargar configuración desde archivo JSON
with open('config.json', 'r', encoding='utf-8') as f:
    config = json.load(f)

# Extraer parámetros
L_x = config['campo']['dimension_x_m']
L_y = config['campo']['dimension_y_m']
R = config['sensor']['rango_cobertura_m']
C_x = config['discretizacion']['celda_x_m']
C_y = config['discretizacion']['celda_y_m']
A_total = config['campo']['area_total_m2']

print("=" * 60)
print("PARÁMETROS DEL PROBLEMA")
print("=" * 60)
print(f"Área del campo: {A_total:,.2f} m²")
print(f"Dimensiones del campo: {L_x} m × {L_y} m")
print(f"Rango de cobertura LoRa: {R} m")
print(f"Tamaño de celda: {C_x} m × {C_y} m")
print("=" * 60)

## 2. Generación del Conjunto J (Puntos de Demanda)

In [ ]:
# Generar puntos centrales de las celdas (Conjunto J)
J = []
j_index = 0

# Comenzar en (C_x/2, C_y/2) y avanzar en pasos de C_x y C_y
y = C_y / 2
while y < L_y:
    x = C_x / 2
    while x < L_x:
        J.append((j_index, x, y))
        j_index += 1
        x += C_x
    y += C_y

# Convertir a array de numpy para facilitar cálculos
J_coords = np.array([(x, y) for _, x, y in J])

print(f"Número de puntos de demanda (|J|): {len(J)}")
print(f"\nPrimeros 5 puntos de demanda:")
for i in range(min(5, len(J))):
    idx, x, y = J[i]
    print(f"  J[{idx}] = ({x:.1f}, {y:.1f})")
print(f"\nÚltimos 3 puntos de demanda:")
for i in range(max(0, len(J)-3), len(J)):
    idx, x, y = J[i]
    print(f"  J[{idx}] = ({x:.1f}, {y:.1f})")

## 3. Generación del Conjunto I (Posibles Ubicaciones de Sensores)

In [ ]:
# Para este problema, asumimos que I = J
# (las posibles ubicaciones coinciden con los centros de las celdas)
I = J.copy()
I_coords = J_coords.copy()

print(f"Número de posibles ubicaciones de sensores (|I|): {len(I)}")
print(f"Nota: En este caso, I = J (ubicaciones coinciden con centros de celdas)")

## 4. Cálculo de la Matriz de Cobertura Binaria $a_{ij}$

In [ ]:
def calcular_distancia(p1, p2):
    """Calcula la distancia euclidiana entre dos puntos."""
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

# Crear matriz de cobertura a_ij
print("Calculando matriz de cobertura...")
n_I = len(I)
n_J = len(J)
a = np.zeros((n_I, n_J), dtype=int)

for i in range(n_I):
    for j in range(n_J):
        dist = calcular_distancia(I_coords[i], J_coords[j])
        if dist <= R:
            a[i, j] = 1

print(f"Matriz de cobertura calculada: {n_I} × {n_J}")
print(f"\nEstadísticas de la matriz de cobertura:")
print(f"  Total de elementos: {n_I * n_J:,}")
print(f"  Elementos con cobertura (a_ij = 1): {np.sum(a):,}")
print(f"  Densidad de cobertura: {100 * np.sum(a) / (n_I * n_J):.2f}%")

# Mostrar ejemplo de una fila de la matriz
print(f"\nEjemplo - Fila 0 (ubicación I[0] = {I_coords[0]}):")
puntos_cubiertos = np.where(a[0] == 1)[0]
print(f"  Cubre {len(puntos_cubiertos)} puntos de demanda")
print(f"  Primeros 10 índices cubiertos: {puntos_cubiertos[:10].tolist()}")

# Verificar que cada punto de demanda puede ser cubierto por al menos un sensor
puntos_sin_cobertura = np.where(np.sum(a, axis=0) == 0)[0]
if len(puntos_sin_cobertura) > 0:
    print(f"\n⚠️ ADVERTENCIA: {len(puntos_sin_cobertura)} puntos de demanda no pueden ser cubiertos")
else:
    print(f"\n✓ Todos los puntos de demanda pueden ser cubiertos por al menos un sensor")

## 5. Formulación del Problema de PLE con PuLP

In [ ]:
print("Formulando el problema de optimización...\n")

# Crear el problema de minimización
prob = LpProblem("Set_Cover_Sensor_Optimization", LpMinimize)

# Variables de decisión: x_i ∈ {0, 1} para cada ubicación i ∈ I
x = {}
for i in range(n_I):
    x[i] = LpVariable(f"x_{i}", cat='Binary')

# Función objetivo: Minimizar Σ x_i
prob += lpSum([x[i] for i in range(n_I)]), "Minimizar_numero_de_sensores"

# Restricciones de cobertura: Σ a_ij * x_i ≥ 1 para todo j ∈ J
for j in range(n_J):
    prob += lpSum([a[i, j] * x[i] for i in range(n_I)]) >= 1, f"Cobertura_punto_{j}"

print("Problema formulado:")
print(f"  Variables de decisión: {n_I}")
print(f"  Restricciones de cobertura: {n_J}")
print(f"  Función objetivo: Minimizar Σ x_i")

## 6. Resolución del Problema y Extracción de Resultados

In [ ]:
print("Resolviendo el problema de optimización...\n")
print("Esto puede tomar algunos minutos dependiendo del tamaño del problema.\n")

# Resolver el problema
inicio = datetime.now()
prob.solve(PULP_CBC_CMD(msg=1))
fin = datetime.now()
tiempo_resolucion = (fin - inicio).total_seconds()

# Verificar el estado de la solución
print("\n" + "=" * 60)
print("RESULTADOS DE LA OPTIMIZACIÓN")
print("=" * 60)
print(f"Estado de la solución: {LpStatus[prob.status]}")
print(f"Tiempo de resolución: {tiempo_resolucion:.2f} segundos")

if prob.status == LpStatusOptimal:
    # Extraer la solución óptima
    N_optimo = int(value(prob.objective))
    sensores_optimos = []
    
    for i in range(n_I):
        if x[i].varValue == 1:
            sensores_optimos.append(i)
    
    # Obtener coordenadas de los sensores óptimos
    coordenadas_optimas = [(I_coords[i][0], I_coords[i][1]) for i in sensores_optimos]
    
    print(f"\n✓ Solución óptima encontrada")
    print(f"\nN_óptimo = {N_optimo} sensores")
    print(f"\nCoordenadas de los sensores óptimos:")
    print("-" * 40)
    for idx, sensor_i in enumerate(sensores_optimos, 1):
        x_coord, y_coord = coordenadas_optimas[idx-1]
        print(f"  Sensor {idx:2d}: ({x_coord:6.1f}, {y_coord:6.1f}) [índice I={sensor_i}]")
    
    # Calcular métricas adicionales
    densidad_sensores = N_optimo / (A_total / 10000)  # sensores por hectárea
    area_por_sensor = A_total / N_optimo
    
    print(f"\nMétricas adicionales:")
    print(f"  Densidad de sensores: {densidad_sensores:.2f} sensores/hectárea")
    print(f"  Área promedio por sensor: {area_por_sensor:,.2f} m²")
    print(f"  Porcentaje de ubicaciones utilizadas: {100*N_optimo/n_I:.2f}%")
    
else:
    print(f"\n⚠️ No se encontró una solución óptima")
    print(f"Estado: {LpStatus[prob.status]}")
    N_optimo = None
    sensores_optimos = []
    coordenadas_optimas = []

## 7. Visualización con Matplotlib

In [ ]:
if N_optimo is not None:
    fig, ax = plt.subplots(figsize=(14, 12))
    
    # Configurar límites del gráfico
    ax.set_xlim(-10, L_x + 10)
    ax.set_ylim(-10, L_y + 10)
    ax.set_aspect('equal')
    
    # Dibujar el contorno del campo
    ax.add_patch(plt.Rectangle((0, 0), L_x, L_y, 
                                fill=False, edgecolor='black', linewidth=2))
    
    # Dibujar grid de celdas si está configurado
    if config['visualizacion']['mostrar_grid']:
        for x_grid in np.arange(0, L_x + C_x, C_x):
            ax.axvline(x=x_grid, color='gray', linewidth=0.3, alpha=0.3)
        for y_grid in np.arange(0, L_y + C_y, C_y):
            ax.axhline(y=y_grid, color='gray', linewidth=0.3, alpha=0.3)
    
    # Dibujar puntos de demanda (todos)
    ax.scatter(J_coords[:, 0], J_coords[:, 1], 
               c='lightgray', s=10, alpha=0.5, label='Puntos de demanda')
    
    # Dibujar círculos de cobertura si está configurado
    if config['visualizacion']['mostrar_circulos_cobertura']:
        for x_s, y_s in coordenadas_optimas:
            circle = Circle((x_s, y_s), R, 
                          color=config['visualizacion']['color_cobertura'], 
                          alpha=config['visualizacion']['alpha_cobertura'])
            ax.add_patch(circle)
    
    # Dibujar sensores óptimos
    sensores_x = [coord[0] for coord in coordenadas_optimas]
    sensores_y = [coord[1] for coord in coordenadas_optimas]
    ax.scatter(sensores_x, sensores_y, 
               c=config['visualizacion']['color_sensores'], 
               s=200, marker='*', 
               edgecolors='black', linewidth=1.5,
               label=f'Sensores LoRa (N={N_optimo})', zorder=5)
    
    # Agregar números a los sensores
    for idx, (x_s, y_s) in enumerate(coordenadas_optimas, 1):
        ax.text(x_s, y_s - 8, str(idx), 
               ha='center', va='top', fontsize=8, fontweight='bold')
    
    # Configurar título y etiquetas
    ax.set_xlabel('X (metros)', fontsize=12)
    ax.set_ylabel('Y (metros)', fontsize=12)
    titulo = f"{config['visualizacion']['titulo']}\n"
    titulo += f"N_óptimo = {N_optimo} sensores | "
    titulo += f"Campo: {L_x}m × {L_y}m | Rango: {R}m"
    ax.set_title(titulo, fontsize=14, fontweight='bold', pad=20)
    ax.legend(loc='upper right', fontsize=10)
    ax.grid(True, alpha=0.2)
    
    plt.tight_layout()
    plt.savefig('distribucion_sensores_optima.png', dpi=300, bbox_inches='tight')
    print("\n✓ Visualización guardada como 'distribucion_sensores_optima.png'")
    plt.show()
else:
    print("No se puede generar visualización sin solución óptima")

## 8. Generación de Reporte Detallado

In [ ]:
if config['reporte']['generar_reporte_detallado'] and N_optimo is not None:
    nombre_reporte = config['reporte']['nombre_archivo']
    
    with open(nombre_reporte, 'w', encoding='utf-8') as f:
        f.write("="*80 + "\n")
        f.write("REPORTE DE OPTIMIZACIÓN DE UBICACIÓN DE SENSORES LoRa\n")
        f.write("Set Cover Problem - Programación Lineal Entera\n")
        f.write("="*80 + "\n\n")
        
        f.write(f"Fecha de generación: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        f.write("-"*80 + "\n")
        f.write("1. PARÁMETROS DEL PROBLEMA\n")
        f.write("-"*80 + "\n")
        f.write(f"Área total del campo: {A_total:,.2f} m²\n")
        f.write(f"Dimensiones del campo: {L_x} m × {L_y} m\n")
        f.write(f"Rango de cobertura del sensor LoRa (R): {R} m\n")
        f.write(f"Tamaño de celda de discretización: {C_x} m × {C_y} m\n")
        f.write(f"Número de puntos de demanda (|J|): {len(J)}\n")
        f.write(f"Número de posibles ubicaciones (|I|): {len(I)}\n\n")
        
        f.write("-"*80 + "\n")
        f.write("2. FORMULACIÓN MATEMÁTICA\n")
        f.write("-"*80 + "\n")
        f.write("Función Objetivo:\n")
        f.write("  Minimizar: Σ x_i  (para todo i ∈ I)\n\n")
        f.write("Restricciones:\n")
        f.write("  Σ a_ij * x_i ≥ 1  (para todo j ∈ J)\n")
        f.write("  x_i ∈ {0, 1}      (para todo i ∈ I)\n\n")
        f.write(f"Donde a_ij = 1 si dist(i,j) ≤ {R}m, 0 en caso contrario\n\n")
        
        f.write("-"*80 + "\n")
        f.write("3. RESULTADOS DE LA OPTIMIZACIÓN\n")
        f.write("-"*80 + "\n")
        f.write(f"Estado de la solución: {LpStatus[prob.status]}\n")
        f.write(f"Tiempo de resolución: {tiempo_resolucion:.2f} segundos\n")
        f.write(f"Solver utilizado: PULP_CBC_CMD\n\n")
        f.write(f"N_óptimo (número mínimo de sensores): {N_optimo}\n\n")
        
        f.write("-"*80 + "\n")
        f.write("4. COORDENADAS DE LOS SENSORES ÓPTIMOS\n")
        f.write("-"*80 + "\n")
        f.write(f"{'Sensor':<10} {'X (m)':<12} {'Y (m)':<12} {'Índice I':<10}\n")
        f.write("-"*80 + "\n")
        for idx, sensor_i in enumerate(sensores_optimos, 1):
            x_coord, y_coord = coordenadas_optimas[idx-1]
            f.write(f"{idx:<10} {x_coord:<12.2f} {y_coord:<12.2f} {sensor_i:<10}\n")
        f.write("\n")
        
        f.write("-"*80 + "\n")
        f.write("5. MÉTRICAS Y ESTADÍSTICAS\n")
        f.write("-"*80 + "\n")
        f.write(f"Densidad de sensores: {densidad_sensores:.4f} sensores/hectárea\n")
        f.write(f"Área promedio cubierta por sensor: {area_por_sensor:,.2f} m²\n")
        f.write(f"Porcentaje de ubicaciones utilizadas: {100*N_optimo/n_I:.2f}%\n")
        f.write(f"Área teórica de cobertura por sensor: {np.pi * R**2:,.2f} m²\n")
        f.write(f"Factor de solapamiento: {(N_optimo * np.pi * R**2) / A_total:.2f}\n\n")
        
        f.write("-"*80 + "\n")
        f.write("6. ANÁLISIS DE COBERTURA\n")
        f.write("-"*80 + "\n")
        
        # Calcular cuántos sensores cubren cada punto de demanda
        cobertura_por_punto = np.sum(a[sensores_optimos, :], axis=0)
        f.write(f"Cobertura mínima por punto: {np.min(cobertura_por_punto)}\n")
        f.write(f"Cobertura máxima por punto: {np.max(cobertura_por_punto)}\n")
        f.write(f"Cobertura promedio por punto: {np.mean(cobertura_por_punto):.2f}\n")
        
        # Distribución de cobertura
        f.write("\nDistribución de cobertura:\n")
        for k in range(1, int(np.max(cobertura_por_punto)) + 1):
            puntos_con_k_sensores = np.sum(cobertura_por_punto == k)
            porcentaje = 100 * puntos_con_k_sensores / n_J
            f.write(f"  Puntos cubiertos por {k} sensor(es): {puntos_con_k_sensores} ({porcentaje:.1f}%)\n")
        
        f.write("\n")
        f.write("="*80 + "\n")
        f.write("FIN DEL REPORTE\n")
        f.write("="*80 + "\n")
    
    print(f"\n✓ Reporte detallado guardado como '{nombre_reporte}'")
    
    # Mostrar un resumen del análisis de cobertura
    print("\n" + "="*60)
    print("ANÁLISIS DE COBERTURA")
    print("="*60)
    print(f"Cobertura mínima por punto: {np.min(cobertura_por_punto)}")
    print(f"Cobertura máxima por punto: {np.max(cobertura_por_punto)}")
    print(f"Cobertura promedio por punto: {np.mean(cobertura_por_punto):.2f}")
    print("\nDistribución:")
    for k in range(1, min(6, int(np.max(cobertura_por_punto)) + 1)):
        puntos_con_k_sensores = np.sum(cobertura_por_punto == k)
        porcentaje = 100 * puntos_con_k_sensores / n_J
        print(f"  {k} sensor(es): {puntos_con_k_sensores} puntos ({porcentaje:.1f}%)")

## 9. Resumen Final (Salida a Consola)

In [ ]:
if N_optimo is not None:
    print("\n" + "="*80)
    print("RESUMEN EJECUTIVO")
    print("="*80)
    print(f"\n🎯 RESULTADO ÓPTIMO: Se requieren {N_optimo} sensores LoRa")
    print(f"\n📊 COBERTURA: 100% del campo ({A_total:,.2f} m²)")
    print(f"\n⏱️  TIEMPO DE CÁLCULO: {tiempo_resolucion:.2f} segundos")
    print(f"\n📍 DENSIDAD: {densidad_sensores:.2f} sensores por hectárea")
    print(f"\n📁 ARCHIVOS GENERADOS:")
    print("   - distribucion_sensores_optima.png (visualización)")
    if config['reporte']['generar_reporte_detallado']:
        print(f"   - {config['reporte']['nombre_archivo']} (reporte detallado)")
    print("\n" + "="*80)
    print("\n✅ OPTIMIZACIÓN COMPLETADA EXITOSAMENTE")
    print("="*80)
else:
    print("\n⚠️ No se pudo completar la optimización")